In [1]:
!pip install torch torchaudio librosa jiwer pandas numpy tqdm
import os
import glob
import pandas as pd
import numpy as np
import librosa
import torch
import torchaudio
from tqdm import tqdm
from jiwer import wer



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.7 MB/s eta 0:00:00


In [5]:
import os, glob

tsvs = glob.glob(EXTRACT_DIR + "/**/*.tsv", recursive=True)

print("FOUND TSV FILES:")
for t in tsvs:
    print(os.path.basename(t))


FOUND TSV FILES:


In [10]:


import os, json, subprocess, textwrap



# Put your key here for private notebook use, or set it in Colab env variables:
os.environ["MOZ_API_KEY"] = "354e3416db01207d8792e39bafc0078b5e228e64bdda21fecbc8c012e8f1e7a5"

assert "MOZ_API_KEY" in os.environ and os.environ["MOZ_API_KEY"].strip(), "Set MOZ_API_KEY env var first!"
MOZ_API_KEY = os.environ["MOZ_API_KEY"].strip()

DATASET_ID = "cmflnuzw71qkz8x3kil3tgjvk"
OUT_TAR = "common_voice_tr_scripted_23_0.tar.gz"

# 1) Create download session -> get token
cmd1 = f'''
curl -s -X POST "https://datacollective.mozillafoundation.org/api/datasets/{DATASET_ID}/download" \
  -H "Authorization: Bearer {MOZ_API_KEY}" \
  -H "Content-Type: application/json"
'''
resp = subprocess.check_output(cmd1, shell=True).decode("utf-8")
print(resp)

data = json.loads(resp)
download_token = data.get("download_token") or data.get("token") or data.get("downloadToken")
assert download_token, f"Could not find download token in response: {data}"
print("DOWNLOAD TOKEN:", download_token)

# 2) Download the dataset tar.gz
cmd2 = f'''
curl -L -X GET "https://datacollective.mozillafoundation.org/api/datasets/{DATASET_ID}/download/{download_token}" \
  -H "Authorization: Bearer {MOZ_API_KEY}" \
  -o "{OUT_TAR}"
'''
subprocess.check_call(cmd2, shell=True)

print("Downloaded:", OUT_TAR, "size(bytes)=", os.path.getsize(OUT_TAR))

{"downloadToken":"dlt_8deec491-0a90-498a-ba90-b5f5648179de","downloadUrl":"https://datacollective.mozillafoundation.org/api/datasets/cmflnuzw71qkz8x3kil3tgjvk/download/dlt_8deec491-0a90-498a-ba90-b5f5648179de","expiresAt":"2025-12-15T11:36:35.781Z","sizeBytes":"2933767497","contentType":"application/gzip","filename":"mcv-scripted-tr-v23.0.tar.gz","checksum":null}
DOWNLOAD TOKEN: dlt_8deec491-0a90-498a-ba90-b5f5648179de
Downloaded: common_voice_tr_scripted_23_0.tar.gz size(bytes)= 2933767497


In [11]:
import tarfile

EXTRACT_DIR = "/content/cv_tr"
os.makedirs(EXTRACT_DIR, exist_ok=True)

with tarfile.open(OUT_TAR, "r:gz") as tar:
    tar.extractall(EXTRACT_DIR)

print("✅ Extracted")


/tmp/ipython-input-927652314.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(EXTRACT_DIR)


✅ Extracted


In [4]:
EXTRACT_DIR = "/content/cv_tr"
tsvs = glob.glob(EXTRACT_DIR + "/**/*.tsv", recursive=True)
validated_tsv = [t for t in tsvs if "validated.tsv" in t.lower()][0]

df = pd.read_csv(validated_tsv, sep="\t")
df = df[["path", "sentence"]].dropna()

print("Total validated samples:", len(df))


IndexError: list index out of range

In [ ]:
clips_dirs = [d for d in glob.glob(EXTRACT_DIR + "/**/clips", recursive=True)]
assert clips_dirs
CLIPS_DIR = clips_dirs[0]

print("train:", train_tsv)
print("dev:", dev_tsv)
print("test:", test_tsv)
print("clips:", CLIPS_DIR)


In [ ]:
def prepare_df(tsv_path, limit=None):
    df = pd.read_csv(tsv_path, sep="\t")
    df = df[["path", "sentence"]].dropna()
    df["audio"] = df["path"].apply(lambda x: os.path.join(CLIPS_DIR, x))
    if limit:
        df = df.head(limit)
    return df.reset_index(drop=True)

test_df = prepare_df(test_tsv, limit=300)  # hızlı denemek için
print("Test samples:", len(test_df))


In [ ]:
CHARS = list(" abcdefghijklmnopqrstuvwxyzçğıöşü")
char2idx = {c: i+1 for i, c in enumerate(CHARS)}  # 0 blank
idx2char = {i: c for c, i in char2idx.items()}

BLANK = 0
VOCAB_SIZE = len(char2idx) + 1


In [ ]:
def fake_ctc_logits(audio_path, vocab_size, max_time=100):
    wav, sr = librosa.load(audio_path, sr=16000)
    T = min(max_time, max(10, len(wav)//320))
    logits = torch.rand(T, vocab_size)
    return logits


def manual_ctc_decode(logits):
    pred = torch.argmax(logits, dim=-1).tolist()

    decoded = []
    prev = BLANK
    for p in pred:
        if p != prev and p != BLANK:
            decoded.append(idx2char.get(p, ""))
        prev = p

    return "".join(decoded)


In [ ]:
references = []
predictions = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    logits = fake_ctc_logits(row["audio"], VOCAB_SIZE)
    pred_text = manual_ctc_decode(logits)

    references.append(row["sentence"].lower())
    predictions.append(pred_text.lower())


wer_score = wer(references, predictions)
print("Manual CTC Decoding WER:", wer_score)
